In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchsummary import summary
from torchvision import datasets, transforms
from functools import partial
import optuna

In [2]:
class MNISTNet(nn.Module):
    def __init__(self):
        super(MNISTNet, self).__init__()
        self.mnist_model = nn.Sequential(
            nn.Conv2d(1, 8, 3, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(8),
            nn.Dropout(0.1),
            nn.Conv2d(8, 12, 3, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(12),
            nn.Dropout(0.1),
            nn.Conv2d(12, 16, 3, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(0.1),
            nn.Conv2d(16, 32, 3, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(2),
            nn.Dropout(0.25),
            nn.Conv2d(32, 16, 1, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(0.1),
            nn.Conv2d(16, 16, 3, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(0.1),
            nn.Conv2d(16,16,3,bias = False),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.MaxPool2d(2,2),
            nn.Dropout(0.25),
            nn.Conv2d(16,10,3)
            
        )
        
    def forward(self, x):
        x = self.mnist_model(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [3]:
# class Net(nn.Module):
#     def __init__(self):
#         super(Net, self).__init__()
#         self.mnist_classifier = nn.Sequential(
#             nn.Conv2d(1, 8, 3, use_bias=False),
#             nn.ReLU(),
#             nn.BatchNorm2d(8),
#             nn.Dropout(0.1),
#             nn.Conv2d(8, 12, 3, use_bias=False)
#         )

In [4]:
torch.manual_seed(42)
batch_size = 128 

kwargs = {'num_workers': 1, 'pin_memory': True}

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        
        transforms.Normalize((0.1307, ), (0.3081, ))
    ])), batch_size=batch_size, shuffle=True,**kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, 
    transform=transforms.Compose([
        transforms.ToTensor(),
        
        transforms.Normalize((0.1307, ), (0.3081, ))
    ])), batch_size=batch_size, shuffle=True,**kwargs)

In [9]:
from tqdm import tqdm
tqdm = partial(tqdm, position=0, leave=True)
def train(model, device, train_loader, optimizer, epoch):
    train_correct = 0
    train_loss = 0
    model.train()
    for _, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)
        train_correct += pred.eq(target.view_as(pred)).sum().item()
        loss = F.nll_loss(output, target)
        train_loss += loss
        loss.backward()
        optimizer.step()
        for param in optimizer.param_groups:
          lr = param['lr']
    train_loss /= len(train_loader.dataset)

    
    #return 100.0 * train_correct/len(train_loader.dataset)
    # print (f'\nTrain set: Average loss: {train_loss}, Train Accuracy: {train_correct}/{len(train_loader.dataset)} ({100. * train_correct / len(train_loader.dataset)}), learning rate : {lr}')
    
    
    

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for _, (data, target) in enumerate(test_loader):
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    #test_loss /= len(test_loader.dataset)

    return  1.0 * correct / len(test_loader.dataset)

    # print (f'\nTest set: Average loss: {test_loss}, Test Accuracy: {correct}/{len(test_loader.dataset)} ({100. * correct / len(test_loader.dataset)})')

In [6]:
# device = torch.device('cuda')
# model = MNISTNet().to(device)
# summary(model, input_size=(1, 28, 28))

# init_lr = 0.03
# optimizer = optim.Adam(model.parameters(), lr = init_lr)
# def adjust_lr(optimizer, epoch):
#     for param_group in optimizer.param_groups:
#       init_lr = param_group['lr']
#     lr = max(round(init_lr * 1/(1 + np.pi/50 * epoch), 10), 0.0005)
#     for param_group in optimizer.param_groups:
#         param_group['lr'] = lr

In [20]:
 device = torch.device('cuda')
# model = MNISTNet().to(device)

def objective(trial):
    #optimizer_name = trial.suggest_categorical('optimizer', ['Adam', 'SGD'])

    model = MNISTNet().to(device)
    lr = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    for epoch in range(20):
        print(f'EPOCH {epoch + 1} / {20}')
        #adjust_lr(optimizer,epoch)
        train(model, device, train_loader, optimizer, epoch)
        accuracy = test(model, device, test_loader)
        trial.report(accuracy, epoch)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()
    
    return accuracy
        #test(model, device, test_loader)
    

In [45]:
# for epoch in range(20):
#     print(f'EPOCH {epoch + 1} / {20}')
#     adjust_lr(optimizer,epoch)
#     train(model, device, train_loader, optimizer, epoch)
#     test(model, device, test_loader)

### HyperParameter Tuning

In [19]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100, timeout=6000)

pruned_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED]
complete_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]

print('Study Stats :')
print('Number of finished trials :', len(study.trials))
print('Number of Complete trials : ', len(complete_trials))
print('Number of pruned trials : ', len(pruned_trials))

print('Best Trial: ')
trial = study.best_trial
print(' Value: ', trial.value)
print('Parameters :')

for key, value in trial.params.items():
    print(f'{key}, {value}')

[I 2020-12-19 21:21:18,077] A new study created in memory with name: no-name-9742bd03-31b4-407f-8708-6dcbc4a2c274
EPOCH 1 / 20
EPOCH 2 / 20
EPOCH 3 / 20
EPOCH 4 / 20
EPOCH 5 / 20
EPOCH 6 / 20
EPOCH 7 / 20
EPOCH 8 / 20
EPOCH 9 / 20
EPOCH 10 / 20
EPOCH 11 / 20
EPOCH 12 / 20
EPOCH 13 / 20
EPOCH 14 / 20
EPOCH 15 / 20
EPOCH 16 / 20
EPOCH 17 / 20
EPOCH 18 / 20
EPOCH 19 / 20
EPOCH 20 / 20
[I 2020-12-19 21:26:10,093] Trial 0 finished with value: 0.9954 and parameters: {'lr': 0.000770360505337441}. Best is trial 0 with value: 0.9954.
EPOCH 1 / 20
EPOCH 2 / 20
EPOCH 3 / 20
EPOCH 4 / 20
EPOCH 5 / 20
EPOCH 6 / 20
EPOCH 7 / 20
EPOCH 8 / 20
EPOCH 9 / 20
EPOCH 10 / 20
EPOCH 11 / 20
EPOCH 12 / 20
EPOCH 13 / 20
EPOCH 14 / 20
EPOCH 15 / 20
EPOCH 16 / 20
EPOCH 17 / 20
EPOCH 18 / 20
EPOCH 19 / 20
EPOCH 20 / 20
[I 2020-12-19 21:31:03,018] Trial 1 finished with value: 0.9955 and parameters: {'lr': 0.00035622297147231736}. Best is trial 1 with value: 0.9955.
EPOCH 1 / 20
EPOCH 2 / 20
EPOCH 3 / 20
EPOCH 4 / 2

KeyboardInterrupt: 